In [ ]:
# default_exp core

# Core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *


In [13]:
notebook2script()

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.


In [14]:
#export

import pkgutil
import importlib
import time
import nbdev.imports
from nbdev.export import *

def get_package_name():
    "Returns name of the current package"
    # see logic in https://github.com/fastai/nbdev/blob/master/nbdev/export.py
    return str(nbdev.imports.Config().lib_path).split('/')[-1]

def reload_package_modules(package_name):
    "Dynamically reloads package `package_name`"
    for _, module_name, _ in (pkgutil.iter_modules([package_name])):
        module = importlib.import_module(package_name + '.' + module_name)
        importlib.reload(module)
        
def export_and_reload_all():
    "Converts all notebooks to modules, and reloads all modules"
    if nbdev.imports.in_ipython():
        notebook2script()
        time.sleep(1)
        reload_package_modules(get_package_name())

In [15]:
export_and_reload_all()

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.
